# Оценка релевантности организаций запросам на Яндекс.Картах с помощью LLM-агента

<img src="https://sun9-65.userapi.com/impg/N4y2cxlL7PauAs82tBNFOUAiNctFICWDy4Mbiw/Jiz1fb7NLWU.jpg?size=1080x1080&quality=95&sign=df2786058624d9ccac3ede4d5d056e2f&type=album" width="500" height="500" />


## Описание задачи

Необходимо создать LLM-агента, который будет оценивать релевантность организаций на Яндекс.Картах широким запросам (например, "ресторан с верандой" или "романтичный джаз-бар"). Такие запросы называются *рубричными*: пользователь здесь ищет не конкретную организацию, а идёт в Яндекс.Карты для поиска и выбора мест.

LLM-агент должен будет самостоятельно находить необходимые данные для принятия правильного решения.

Данные представлены компанией Яндекс и являются результатами асессорской разметки релевантности. Мы очень заинтересован в успешном решении задачи:)

## План работы

1. Написать сильный бейзлайн. Например, можно просто делать один запрос в LLM, возможно с добавлением в промпт размеченных примеров. Либо можно дообучить трансформер на задачу классификации.
2. Разобраться с фреймворком для реализации LLM-агентов (предлагается использовать LangGraph, оба примера в доп. материалах используют его).
3. Реализовать первую версию агента: предложить ему не принимать решение мгновенно, а ходить в поисковик для уточнения своего ответа.
4. Поработать над промптами для улучшения качества. Здесь нужно будет посмотреть, в каких запросах агент ошибается. **ПОЖАЛУЙСТА, НЕ ПОДГЛЯДЫВАЙТЕ, КАК ИМЕННО АГЕНТ ОШИБАЕТСЯ НА EVAL-МНОЖЕСТВЕ, ДЛЯ ЭТОГО ЕСТЬ TRAIN**.
5. Другие идеи (опционально):
- с помощью нейросетевого ретривала можно найти в обучающем множестве похожие размеченные примеры
- можно реализовать tool для парсинга сайта, которую агент будет вызывать

## Про платные API

Обратите внимание, что для реализации проекта потребуется самостоятельно ходить в OpenAI API (и другие платные API). На это может уйти существенная сумма (по грубой оценке ментора, 500 запусков агента на стеке OpenAI легко уложатся в 2к рублей, но риски вылезти за эту границу есть. Можно использовать и более дешевые LLM). Также поможет иностранная карта для оплаты этих сервисов, но в целом без нее можно обойтись.

Сам я для запросов в OpenAI API часто использую сервис https://vsegpt.ru/ (не реклама). При их использовании вам не нужен VPN, достаточно российской карты, а цены не сильно отличаются от цен OpenAI, но нужно оплатить подписку (400 рублей).

**О ТОМ, КАКИЕ КЛАССНЫЕ И ДЕШЕВЫЕ АЛЬТЕРНАТИВЫ ВЫ НАШЛИ, ОБЯЗАТЕЛЬНО НАПИШИТЕ В ЧАТ**

## Система оценивания

1. Реализация бейзлайна (без использования агента, но можно с использованием LLM) -- 2 балла
2. Реализация "какого-то" агента, который под капотом имеет возможность обращаться к поисковой строке для уточнения результата -- 4 балла
3. Существенно побить бейзлайн или безуспешно приложить к этому усилия и "показать", что агенты в этой задаче прироста не дают -- 4 балла
4. (опционально, за шоколадку) реализовать одну из доп. идей по улучшению агента

## Что почитать/посмотреть по теме

1. Статья, по которой можно составить какое-то представление об агенте -- https://habr.com/ru/articles/864646/
2. https://www.youtube.com/watch?v=U6LbW2IFUQw -- неплохое видео, где на практике подробно расписано, как агента можно создавать
3. Придётся самим много читать и разбираться. Делитесь идеями в чате:)

## Описание и загрузка данных

Данные: https://disk.yandex.ru/d/6d5hFHvpAZjQdw

Ваша задача -- предсказать колонку relevance, используя все остальные данные об организации. Загрузим данные и посмотрим на них

In [2]:
import pandas as pd
data = pd.read_json(path_or_buf=r"D:\ML\Data\dls2_prj\data_final_for_dls.jsonl", lines=True)

In [3]:
data.head()

,Text,address,name,normalized_main_rubric_name_ru,permalink,prices_summarized,relevance,reviews_summarized
0,сигары,"Москва, Дубравная улица, 34/29",Tabaccos; Магазин Tabaccos; Табаккос,Магазин табака и курительных принадлежностей,1263329400,None,1.0,"Организация занимается продажей табака, курите..."
1,кальянная спб мероприятия,"Санкт-Петербург, Большой проспект Петроградско...",PioNero; Pionero; Пицца Паста бар; Pio Nero; P...,Кафе,228111266197,PioNero предлагает разнообразные блюда итальян...,0.0,"Организация PioNero — это кафе, бар и ресторан..."
2,Эпиляция,"Московская область, Одинцово, улица Маршала Жу...",MaxiLife; Центр красоты и здоровья MaxiLife; Ц...,Стоматологическая клиника,1247255817,"Стоматологическая клиника, массажный салон и к...",1.0,"Организация занимается стоматологическими, кос..."
3,спортзал где 1 занятие бесплатно,"Москва, Страстной бульвар, 13А",Унца Унца Спорт; Unza Unza Sport,Фитнес-клуб,201938477844,Фитнес-клуб предлагает пробные занятия по разл...,0.1,Организация «Унца Унца Спорт» предоставляет ус...
4,стиральных машин,"Москва, улица Обручева, 34/63",М.Видео; M Video; M. Видео; M.Видео; Mvideo; М...,Магазин бытовой техники,1074529324,М.Видео предлагает широкий ассортимент бытовой...,1.0,Организация занимается продажей бытовой техник...


In [4]:
data['relevance'].value_counts()

relevance
1.0    15882
0.0    14509
0.1     4703
Name: count, dtype: int64

Здесь 1.0 соответствует оценке RELEVANT_PLUS, 0.1 -- оценке RELEVANT_MINUS, 0.0 -- оценке IRRELEVANT.

Ваша задача -- построить LLM-агента, который будет предсказывать релевантность.

Выделим данные для оценки качества агента. Запуск агента -- это тяжелая и потенциально дорогая операция. Поэтому eval-множество имеет размер 500. Также для простоты из eval-множества выкинуты данные с оценкой RELEVANT_MINUS. Тем не менее, вы можете использовать такие примеры для подачи примеров агенту.

**ОБРАТИТЕ ВНИМАНИЕ, ЧТО В EVAL-ДАННЫЕ НЕЛЬЗЯ ПОДГЛЯДЫВАТЬ ДЛЯ КАЛИБРОВКИ АГЕНТА!!! ДЛЯ ЭТОГО ЕСТЬ ОБУЧАЮЩИЕ ДАННЫЕ**

В качестве метрики качества мы будем использовать обычную ACCURACY, поскольку классы сбалансированы.

In [ ]:
train_data = data[570:]
eval_data = data[:570]
eval_data = eval_data[eval_data["relevance"] != 0.1]
eval_data

,Text,address,name,normalized_main_rubric_name_ru,permalink,prices_summarized,relevance,reviews_summarized
0,сигары,"Москва, Дубравная улица, 34/29",Tabaccos; Магазин Tabaccos; Табаккос,Магазин табака и курительных принадлежностей,1263329400,None,1.0,"Организация занимается продажей табака, курите..."
1,кальянная спб мероприятия,"Санкт-Петербург, Большой проспект Петроградско...",PioNero; Pionero; Пицца Паста бар; Pio Nero; P...,Кафе,228111266197,PioNero предлагает разнообразные блюда итальян...,0.0,"Организация PioNero — это кафе, бар и ресторан..."
2,Эпиляция,"Московская область, Одинцово, улица Маршала Жу...",MaxiLife; Центр красоты и здоровья MaxiLife; Ц...,Стоматологическая клиника,1247255817,"Стоматологическая клиника, массажный салон и к...",1.0,"Организация занимается стоматологическими, кос..."
4,стиральных машин,"Москва, улица Обручева, 34/63",М.Видео; M Video; M. Видео; M.Видео; Mvideo; М...,Магазин бытовой техники,1074529324,М.Видео предлагает широкий ассортимент бытовой...,1.0,Организация занимается продажей бытовой техник...
5,сеть быстрого питания,"Санкт-Петербург, 1-я Красноармейская улица, 15",Rostic's; KFC; Ресторан быстрого питания KFC,Быстрое питание,1219173871,Rostic's предлагает различные наборы быстрого ...,1.0,"Организация занимается быстрым питанием, предо..."
...,...,...,...,...,...,...,...,...
561,наращивание ресниц,"Саратов, улица имени А.С. Пушкина, 1",Сила; Sila; Beauty brow; Студия бровей Beauty ...,Салон красоты,236976975812,Салон красоты «Сила» предлагает услуги по уход...,1.0,Организация «Сила» занимается предоставлением ...
565,игры,"Москва, Щёлковское шоссе, 79, корп. 1",YouPlay; YouPlay КиберКлуб,Компьютерный клуб,109673025161,YouPlay КиберКлуб предлагает услуги по игре на...,0.0,Организация занимается предоставлением услуг к...
566,домашний интернет в курске что подключить отзы...,"Курск, Садовая улица, 5",Цифровой канал; Digital Channel; DChannel; ЦК;...,Телекоммуникационная компания,1737991898,None,0.0,None
567,гостиница волгодонск сауна номер телефона,"Ростовская область, городской округ Волгодонск...",Поплавок; Poplavok,"База , дом отдыха",147783493467,"Предлагает размещение в различных типах жилья,...",0.0,Организация «Поплавок» предлагает услуги базы ...


In [ ]:
eval_data.to_excel("eval_data.xlsx")